In [37]:
# importing the tensorflow package
import tensorflow as tf

import numpy as np
from keras import optimizers
from keras.layers import SimpleRNN, LSTM, GRU

from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation
from keras.datasets import imdb

In [38]:
tf.test.is_built_with_cuda()

True

In [39]:
tf.config.list_physical_devices('GPU')

[]

# IMDB reviews (keras)

In [40]:
max_features = 25000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [41]:
# Retrieve the word index file mapping words to indices
word_index = imdb.get_word_index()

# Reverse the word index to obtain a dict mapping indices to words
inverted_word_index = dict((i, word) for (word, i) in word_index.items())

# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[0])

decoded_sequence

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when fro

In [42]:
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

25000 тренировочные последовательности
25000 тестовые последовательности


In [44]:
# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 200
batch_size = 25 # увеличьте значение для ускорения обучения
out_dim = 256
dropout = 0.12
epochs = 1


x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 200)
x_test shape: (25000, 200)


In [45]:
model = Sequential()
model.add(Embedding(max_features, out_dim))
model.add(GRU(out_dim, dropout=dropout , recurrent_dropout=dropout ))
model.add(Dense(1, activation='sigmoid'))

In [46]:
opt = optimizers.Adam(learning_rate=0.0005)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [47]:
print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Процесс обучения...
1000/1000 [==============================] - 1375s 1s/step - loss: 0.4366 - accuracy: 0.7903 - val_loss: 0.3511 - val_accuracy: 0.8524


In [48]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

1000/1000 [==============================] - 208s 208ms/step - loss: 0.3511 - accuracy: 0.8524
Результат при тестировании: 0.35109177231788635
Тестовая точность: 0.852400004863739


### Задание 1
Положительный эффект на модель оказали следующие изменения:
- увеличение количества признаков
- увеличение количества слов в комментарии
- уменьшение размера бача
- снижение шумности данных
- сменил модель на GRU

# Генерация текста на основе книжки «Алиса в стране чудес»

In [49]:
# построчное чтение из примера с текстом 
with open("../input/alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set(text)
nb_chars = len(chars)

In [50]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [51]:
# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [52]:
input_chars[0], label_chars[0]

('alices adv', 'e')

In [53]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [54]:
X.shape

(139615, 10, 40)

In [55]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 256
NUM_ITERATIONS = 23 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 2
NUM_PREDS_PER_EPOCH = 250

In [56]:
model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [57]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration+1))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred

Итерация #: 1
Epoch 1/2
1091/1091 [==============================] - 48s 43ms/step - loss: 2.3201
Epoch 2/2
1091/1091 [==============================] - 45s 41ms/step - loss: 1.8707
Генерация из посева: stily. con
stily. cone the mouse the gry how in a dont on the said the gry how in a dont on the said the gry how in a dont on the said the gry how in a dont on the said the gry how in a dont on the said the gry how in a dont on the said the gry how in a dont on the said ==================================================
Итерация #: 2
Epoch 1/2
1091/1091 [==============================] - 45s 41ms/step - loss: 1.6757
Epoch 2/2
1091/1091 [==============================] - 45s 41ms/step - loss: 1.5434
Генерация из посева: of the e--
of the e------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------=

### Задание 2
Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом,
чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший
текст из получившихся и опишите предпринятые для его получения действия. Можно
использовать текст другого произведения.

### отрывок
that was linked into herself, and began to repeat it, but have made a dreadful time, said the caterpillar took the hookah out of its mouth off, but she did not like to be told so. its really dreadful, she muttered to herself, and began to

### Предложения
1. Убрать из текста книги главки
2. Увеличить объем выборки
3. Увеличить размер бачей
4. Увеличить количество итераций и эпох

Мне кажется что чем больше данных будет иметь алгоритм тем больше вероятность построения правильных предложений